In [ ]:
#!pip install selenium

In [ ]:
#!pip install bs4

In [3]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By


from bs4 import BeautifulSoup

import requests
import pandas as pd

In [4]:
site = 'https://wuzzuf.net/jobs/egypt'
# driver = webdriver.Chrome()
driver=webdriver.Firefox()
driver.get(site)
driver.maximize_window()
WebDriverWait(driver,timeout=4)


<selenium.webdriver.support.wait.WebDriverWait (session="4cf1970d-5f49-4692-b1da-ba938a743731")>

## Functions

In [5]:

def filter_tags(tag):
        return tag.has_attr('href') and  tag.get('href').startswith('/jobs/p/')


In [6]:
def get_data(page_content):
    """
    page_content: soup
    """

    
    #get title data
    #1.get title section tag that contain basic info in html page
    title_section = page_content.find_all('section',class_ = 'css-dy1y6u')
    
    #2. get the divs that contain the required data
    divs = title_section[0].find('div',class_='css-bjn8wh' )

    #3. get job title 
    title=divs.find('h1')
    job_title=title.text
    
    #4. get job type
    job_type=title_section[0].find('span')
    Job_type=job_type.text
    
    #5. get company name 
    company=title_section[0].find('a',class_="css-p7pghv")
    
    if company:
        company_name=company.text
    else:
        company=title_section[0].find('div',class_="css-9iujih")
        company_name=company.text
    
    #6. get company location
    location=title_section[0].find('strong',class_='css-9geu3q')
    location.div.decompose()
    Company_location=location.text
    #create dictionary for the previous data 
    d1={ 'job title':[job_title],
       'job_type':[Job_type],
       'company':[company_name],
       'location':[Company_location]

    }
    df1=pd.DataFrame(d1)

    

    ###############################################################
    ##get data of job details section
    #1. get section that contain job details in html page 
    
    sections = page_content.find_all('section',class_ = 'css-3kx5e2')

    divs = sections[0].find_all('div')

    job_details = []
    tr = ['Experience Needed:','Career Level:', 'Education Level:', 'Salary:', 'Job Categories:', 'skills and tools' ]
    for div in divs:
        spans = div.find_all('span')

        temp =  set()
        for s in spans:
            if s.text in tr:
                continue
            temp.add(s.text)

        job_details.append([str(temp).replace('{','').replace('}','')])


    df2= pd.DataFrame(columns=tr)

    
    d2 = dict(zip(tr,job_details))
    
    test_df = pd.DataFrame(data = d2 )
    df2=pd.concat([df2,test_df],axis=0)


    final_df=pd.concat([df1,df2],axis=1)
    return final_df



In [7]:
def jobs_scrap(page_soup):
    ################################################3
    #1.get page links
    
    home = 'https://wuzzuf.net'
    links= [] 
    global jobs_df
    
    for i in page_soup.find_all(filter_tags):
        links.append(home + i.get('href'))
    
    ################################################
    #####2. get data for each job link
    for link in links:
        driver.get(link)
        soup1=BeautifulSoup(driver.page_source, 'lxml')
        df=get_data(soup1)
        
        jobs_df=pd.concat([jobs_df,df],axis=0)
    
    jobs_df.reset_index(inplace=True ,drop=True)
   


    

### iterate

#### 1. Access search box

In [26]:
#Keywords : AI Engineer ,ML Engineer , Data Analyst , Data scientist , Data Engineer


driver.get(site)
search_box = driver.find_element(By.CLASS_NAME,'search-bar-input')

search_box.clear()
search_box.send_keys('Data Engineer'+ Keys.ENTER)


In [27]:
driver.switch_to.window(driver.window_handles[-1])
first_page=driver.current_url

first_page

'https://wuzzuf.net/search/jobs/?q=Data+Engineer&a=hpb'

In [28]:
# soup = BeautifulSoup(driver.page_source, 'lxml')

In [29]:
jobs_df=pd.DataFrame(columns=['job title', 'job_type', 'company', 'location', 'Experience Needed:',\
       'Career Level:', 'Education Level:', 'Salary:', 'Job Categories:',
       'skills and tools'])

In [30]:
jobs_df

,job title,job_type,company,location,Experience Needed:,Career Level:,Education Level:,Salary:,Job Categories:,skills and tools


In [31]:
# jobs_scrap(soup)

In [32]:
###scroll between pages
driver.get(first_page)


buttons=driver.find_element(By.CLASS_NAME,'css-if9uys')
Buttons=buttons.find_elements(By.TAG_NAME,'button')


In [33]:
Buttons[0]

<selenium.webdriver.remote.webelement.WebElement (session="4cf1970d-5f49-4692-b1da-ba938a743731", element="a8376886-8c89-42a2-9bcd-3dbebc82dc33")>

In [34]:
Xpath=f'/html/body/div[1]/div/div[4]/div/div/div[3]/ul/li[{len(Buttons)+1}]/button'
page=first_page
for i in range(len(Buttons)-1):
    print(page)
    driver.get(page)
    
    s1 = BeautifulSoup(driver.page_source, 'lxml')
    
    jobs_scrap(s1)
    
    driver.get(page)
    driver.find_element(By.XPATH,Xpath).click()
    driver.switch_to.window(driver.window_handles[-1])
    page= driver.current_url
    
    
    
    
    


https://wuzzuf.net/search/jobs/?q=Data+Engineer&a=hpb
https://wuzzuf.net/search/jobs/?a=hpb&q=Data%20Engineer&start=1
https://wuzzuf.net/search/jobs/?a=hpb&q=Data%20Engineer&start=4
https://wuzzuf.net/search/jobs/?a=hpb&q=Data%20Engineer&start=5
https://wuzzuf.net/search/jobs/?a=hpb&q=Data%20Engineer&start=6


In [35]:
jobs_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 75 entries, 0 to 74
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   job title           75 non-null     object
 1   job_type            75 non-null     object
 2   company             75 non-null     object
 3   location            75 non-null     object
 4   Experience Needed:  75 non-null     object
 5   Career Level:       75 non-null     object
 6   Education Level:    75 non-null     object
 7   Salary:             75 non-null     object
 8   Job Categories:     75 non-null     object
 9   skills and tools    75 non-null     object
dtypes: object(10)
memory usage: 6.0+ KB


In [36]:
jobs_df.to_csv("Data Engineer.csv", sep=',' ,index = False)

In [49]:
del jobs_df

In [ ]:
# jobs_df.reset_index(drop=True,inplace=True)

In [ ]:
# jobs_df

In [ ]:
# driver.close()